In [9]:
!pip install numpy opencv-python scikit-image matplotlib
import cv2
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from tensorflow.keras.models import load_model

#code wrote by Alan supported by Chatgpt

In [10]:
img = Image.open("D:/Download/TRAIN000000.jpg")
img_array = np.array(img)

def create_circular_mask(h, w, center=None, radius=None):
    if center is None:
        center = (int(w/2), int(h/2))
    if radius is None: 
        radius = min(center[0], center[1], w-center[0], h-center[1])

    Y, X = np.ogrid[:h, :w]
    dist_from_center = np.sqrt((X - center[0])**2 + (Y-center[1])**2)

    mask = dist_from_center <= radius
    return mask

h, w = img_array.shape[:2]
mask = create_circular_mask(h, w)

img_array[~mask] = 0

# Convert the PIL Image object to a NumPy array before saving and displaying
masked_img_array = np.array(img_array)

# Save the image using OpenCV
cv2.imwrite('D:/Download/processed_image.jpg', masked_img_array)

# Display the image using OpenCV
cv2.imshow('Processed Image', masked_img_array)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [12]:
def create_circular_mask(h, w, center=None, radius=None):
    if center is None:
        center = (int(w/2), int(h/2))
    if radius is None: 
        radius = min(center[0], center[1], w-center[0], h-center[1])

    Y, X = np.ogrid[:h, :w]
    dist_from_center = np.sqrt((X - center[0])**2 + (Y-center[1])**2)

    mask = dist_from_center <= radius
    return mask

def preprocess_image(image_path):
    img = Image.open(image_path)
    img_array = np.array(img)

    h, w = img_array.shape[:2]
    mask = create_circular_mask(h, w)

    img_array[~mask] = 0
    masked_img_array = np.array(img_array)
    
    # Ensure the image is in BGR format for OpenCV compatibility
    masked_img_bgr = cv2.cvtColor(masked_img_array, cv2.COLOR_RGB2BGR)
    return masked_img_bgr

def detect_optic_disc_and_cup(img_bgr):
    gray = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2GRAY)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    clahe_img = clahe.apply(gray)
    blurred_img = cv2.GaussianBlur(clahe_img, (5, 5), 0)
    
    _, binary_thresh = cv2.threshold(blurred_img, 70, 255, cv2.THRESH_BINARY)
    
    contours, _ = cv2.findContours(binary_thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    
    if not contours:
        print("No contours found!")
        return None

    optic_disc_contour = max(contours, key=cv2.contourArea)
    optic_disc_mask = np.zeros_like(gray)
    cv2.drawContours(optic_disc_mask, [optic_disc_contour], -1, (255), thickness=-1)
    
    optic_disc_only = cv2.bitwise_and(clahe_img, clahe_img, mask=optic_disc_mask)
    _, optic_cup_mask = cv2.threshold(optic_disc_only, 140, 255, cv2.THRESH_BINARY)
    
    combined_mask = cv2.addWeighted(optic_disc_mask, 1, optic_cup_mask, 1, 0)
    
    return combined_mask

image_path = 'D:/Download/TRAIN000000.jpg'
masked_img_bgr = preprocess_image(image_path)
processed_image = detect_optic_disc_and_cup(masked_img_bgr)

# You would also replace 'output_path' with the actual path where you want to save the processed image.
# if processed_image is not None:
cv2.imwrite('output_path/processed_image.jpg', processed_image)
cv2.imshow('Processed Image', processed_image)
cv2.waitKey(0)
cv2.destroyAllWindows()
# else:
#     print("Processing failed, no contours detected.")